In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
# Define the PINN model as a fully connected neural networ
a = torch.tensor([1.0], device="cuda")
b = torch.tensor([2.0], device="cuda")
c = a + b
print(c)

tensor([3.], device='cuda:0')


In [ ]:

from torch.utils.data import TensorDataset, DataLoader

df = pd.read_csv("../simulation_data/simulation_fixed_v3.csv")
df = df[df["t"] >= 12]
X_data = df[['x', 'y', 't']].values.astype(np.float32)
y_data = df['temperature'].values.astype(np.float32).reshape(-1, 1)

# Optionally, you can normalize the inputs if needed. For example:
# X_data = (X_data - X_data.mean(axis=0)) / X_data.std(axis=0)

# Create PyTorch tensors
inputs_tensor = torch.tensor(X_data)
targets_tensor = torch.tensor(y_data)

# Create a Dataset and DataLoader for batching
dataset = TensorDataset(inputs_tensor, targets_tensor)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

(459840, 4)


In [6]:
a = torch.tensor([1, 2, 3])
b = torch.tensor([4, 5, 6])
torch.dstack((a,b))
a = torch.tensor([[1,3,4],[2,5,6],[3,8,9]])
b = torch.tensor([[4,3,3],[5,32,2],[6,4,11]])
torch.dstack((a,b))
y_plot = torch.linspace(0, 1, 100).reshape(1, 1, -1)
print(y_plot)

tensor([[[0.0000, 0.0101, 0.0202, 0.0303, 0.0404, 0.0505, 0.0606, 0.0707,
          0.0808, 0.0909, 0.1010, 0.1111, 0.1212, 0.1313, 0.1414, 0.1515,
          0.1616, 0.1717, 0.1818, 0.1919, 0.2020, 0.2121, 0.2222, 0.2323,
          0.2424, 0.2525, 0.2626, 0.2727, 0.2828, 0.2929, 0.3030, 0.3131,
          0.3232, 0.3333, 0.3434, 0.3535, 0.3636, 0.3737, 0.3838, 0.3939,
          0.4040, 0.4141, 0.4242, 0.4343, 0.4444, 0.4545, 0.4646, 0.4747,
          0.4848, 0.4949, 0.5051, 0.5152, 0.5253, 0.5354, 0.5455, 0.5556,
          0.5657, 0.5758, 0.5859, 0.5960, 0.6061, 0.6162, 0.6263, 0.6364,
          0.6465, 0.6566, 0.6667, 0.6768, 0.6869, 0.6970, 0.7071, 0.7172,
          0.7273, 0.7374, 0.7475, 0.7576, 0.7677, 0.7778, 0.7879, 0.7980,
          0.8081, 0.8182, 0.8283, 0.8384, 0.8485, 0.8586, 0.8687, 0.8788,
          0.8889, 0.8990, 0.9091, 0.9192, 0.9293, 0.9394, 0.9495, 0.9596,
          0.9697, 0.9798, 0.9899, 1.0000]]])


In [ ]:
class Surrogate(nn.Module):
    def __init__(self, hidden_size=128):
        super(Surrogate, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(3, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 1)
        )
        
    def forward(self, x):
        return self.model(x)
    
model = Surrogate()

# Define loss and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    epoch_loss = 0
    for batch_inputs, batch_targets in dataloader:
        optimizer.zero_grad()
        predictions = model(batch_inputs)
        loss = criterion(predictions, batch_targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(dataloader)}")

In [ ]:
x = torch.linspace(0, 40, 40)
y = torch.linspace(0, 24, 24)
xx, yy = torch.meshgrid([x, y])

x_eval = xx.flatten()
y_eval = yy.flatten()
t = 500 * torch.ones(x_eval.shape)
print(x_eval.shape)
pred = model(torch.vstack([x_eval, y_eval, t]).T)

pred = pred.reshape(40, 24)

plt.figure(figsize=(8, 6))
cp = plt.imshow(pred.detach(), cmap='viridis')
plt.colorbar(cp)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Predicted Solution u(x,t)')
plt.show()


In [ ]:
import torch
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
from IPython.display import HTML

# Define spatial grid
x = torch.linspace(0, 40, 40)
y = torch.linspace(0, 24, 24)
xx, yy = torch.meshgrid([x, y])

# Flatten x and y coordinates
x_eval = xx.flatten()
y_eval = yy.flatten()

# Define time steps
time_steps = torch.linspace(0, 1000, 200)  

# Set up figure and axis
fig, ax = plt.subplots(figsize=(8, 6))
cp = ax.imshow(np.zeros((40, 24)).T, cmap='viridis', vmin=28, vmax=50)  # Initialize empty frame
cbar = plt.colorbar(cp)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Predicted Solution u(x,t)')

# Function to update the frame
def update(frame):
    t = time_steps[frame] * torch.ones(x_eval.shape)  # Update time
    pred = model(torch.vstack([x_eval, y_eval, t]).T)  # Get predictions
    pred = pred.reshape(40, 24).T.detach().numpy()  # Reshape for plotting
    
    cp.set_data(pred.T)
    ax.set_title(f'Predicted Solution u(x,t) at t={int(t[0].item())}')  # Update title

    return [cp]

# Create animation
ani = animation.FuncAnimation(fig, update, frames=len(time_steps), interval=100, blit=False)

# Display animation in Jupyter Notebook
HTML(ani.to_jshtml())
